<a href="https://colab.research.google.com/github/Pheol9166/AIstudy/blob/main/ML/softmax_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multi-class Classification

#### softmax regression
- iris data 사용

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np

In [12]:
iris_data = load_iris(as_frame=True)
X = iris_data.data
target = iris_data.target
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [13]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, target)
X_train[:5]

array([[ 0.55333328,  0.78880759,  1.0469454 ,  1.58046376],
       [ 1.64384411,  1.24920112,  1.33113254,  1.71209594],
       [ 0.79566902, -0.13197948,  0.99010798,  0.79067065],
       [ 0.18982966, -1.97355361,  0.13754657, -0.26238682],
       [-1.50652052,  1.24920112, -1.56757623, -1.3154443 ]])

In [15]:
# 모델 제작

class SoftmaxRegression():
  def __init__(self, learning_rate=0.01, iterations=1000, batch_size=32, epoch= 10000):
    self._learning_rate = learning_rate
    self._iterations = iterations
    self._batch_size = batch_size
    self._epoch  = epoch
    self._W = None
    self._B = None

  def softmax(self, z):
    max_val = np.max(z) # overflow 방지
    exp_z = np.exp(z - max_val)
    return exp_z / np.sum(exp_z)

  def one_hot_encoding(self, y, num_classes):
    one_hot = np.zeros((len(y), num_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

  def loss(self, h, y):
    if y.ndim == 1:
      h = h.reshape(1, h.size)
      y = y.reshape(1, y.size)

    delta = 1e-7
    return -np.sum(h * np.log(y + delta)) / self._batch_size

  def fit(self, X, y):
    num_samples, num_features = X.shape
    num_classes = len(np.unique(y))

    y_one_hot = self.one_hot_encoding(y, num_classes)

    self._W = np.random.rand(num_features, num_classes)
    self._B = np.zeros((1, num_classes))

    num_batches = num_samples // self._batch_size

    for epoch in range(self._epoch):
      for batch in range(num_batches):
        start = batch * self._batch_size
        end = (batch + 1) * self._batch_size
        X_batch = X[start : end]
        y_batch = y_one_hot[start : end]

        logits = np.dot(X_batch, self._W) + self._B
        predictions = self.softmax(logits)

        loss = self.loss(predictions, y_batch)

        gradient_weights = np.dot(X_batch.transpose(), (predictions - y_batch)) / self._batch_size
        gradient_bias = np.sum(predictions - y_batch, axis=0, keepdims=True) / self._batch_size

        self._W -= self._learning_rate * gradient_weights
        self._B -= self._learning_rate * gradient_bias

      print(f"Epoch {epoch + 1} / {self._epoch}, Loss: {loss}")

  def predict(self, X):
    logits = np.dot(X, self._W) + self._B
    predictions = self.softmax(logits)
    result = np.argmax(predictions, axis=1)
    return result

In [16]:
model = SoftmaxRegression(learning_rate=0.01, iterations=1000, batch_size=32, epoch= 100)
model.fit(X_train, y_train)

Epoch 1 / 100, Loss: 0.32908354858494837
Epoch 2 / 100, Loss: 0.3244010346726045
Epoch 3 / 100, Loss: 0.3197001651104133
Epoch 4 / 100, Loss: 0.3149839006660843
Epoch 5 / 100, Loss: 0.31025516777775747
Epoch 6 / 100, Loss: 0.3055168557291648
Epoch 7 / 100, Loss: 0.3007718140928432
Epoch 8 / 100, Loss: 0.29602285040149295
Epoch 9 / 100, Loss: 0.2912727280106382
Epoch 10 / 100, Loss: 0.28652416411923787
Epoch 11 / 100, Loss: 0.2817798279186929
Epoch 12 / 100, Loss: 0.27704233884465035
Epoch 13 / 100, Loss: 0.272314264910014
Epoch 14 / 100, Loss: 0.267598121101542
Epoch 15 / 100, Loss: 0.26289636782624587
Epoch 16 / 100, Loss: 0.2582114093974466
Epoch 17 / 100, Loss: 0.253545592553734
Epoch 18 / 100, Loss: 0.24890120500716376
Epoch 19 / 100, Loss: 0.24428047401980635
Epoch 20 / 100, Loss: 0.239685565010181
Epoch 21 / 100, Loss: 0.23511858019319407
Epoch 22 / 100, Loss: 0.2305815572589202
Epoch 23 / 100, Loss: 0.22607646809695195
Epoch 24 / 100, Loss: 0.22160521757408555
Epoch 25 / 100, Lo

In [17]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 2, 2, 1, 0, 0, 2, 2, 0])

In [18]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[18,  0,  0],
       [ 1,  4,  5],
       [ 0,  1,  9]])

In [19]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8157894736842105

In [20]:
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.8284527107241788, 0.8157894736842105, 0.7986012328117592, None)